# Hamiltonian

The core mechanical quantities of a chemistry system is the Hamiltonian. Hamiltonian operator should include the kinetic energy and potential energy terms of all atomic nuclei and all electrons. It is generally assumed that the molecule is in a vacuum and adiabatic state in isolation. At this time, the interaction potential energy between the nucleus and the electron in the molecule is only related to distance from each other and time independent. Its expression is:
\begin{equation}
\begin{aligned}
\hat{H}= &-\sum^N_{i=1}\frac{\hbar^2}{2m_i}{\nabla}_i^2
-\sum^N_{i=1}\sum^M_{\alpha=1} \frac{Z_a e^2}{\textbf{r}_{ia}}\\
&+\sum^N_{i=1}\sum^N_{j>i} \frac{e^2}{\textbf{r}_{ij}}
+\sum^N_{a=1}\sum^M_{b=1} \frac{Z_a Z_b e^2}{\textbf{R}_{ab}}
\end{aligned}
\tag{1}
\label{ham}
\end{equation}

Where $m_i$ is the mass of electron. $M_\alpha$ and $Z_\alpha$ refer to the mass and charge of atomic nucleus. $R_{\alpha\beta}$, $r_{i\alpha}$ and $r_{ij}$ is the distance between two nucleus, atomic nuclei and electron and two electrons respectively. The explicit representation of Laplacian operator is:
\begin{equation}
\boldsymbol{\nabla}^2 = \frac{\partial^2}{\partial x^2} +\frac{\partial^2}{\partial y^2} 
+\frac{\partial^2}{\partial z^2}
\end{equation}

To build a Hamiltonian object, MoHa need both molecular geometry and basis object. Only instance of `Operator` class are allowed to be the element of `Hamiltonian` instances. 

|  Name  | Operators |Access Integral | Shape |
|:--------:|:--------:|:------:|:------:|
|Nuclear Repuslion| $\sum^N_{a=1}\sum^M_{b=1} \frac{Z_a Z_b e^2}{\textbf{R}_{ab}}$  | Nuc   | 1   | 
|Overlap| 1 |  S   | (nbasis,nbasis)   | 
|Kinetic| $-\sum^N_{i=1}\frac{\hbar^2}{2m_i}{\nabla}_i^2$ |  T   | (nbasis,nbasis) | 
|Nuclear Attraction| $-\sum^N_{i=1}\sum^M_{\alpha=1} \frac{Z_a e^2}{\textbf{r}_{ia}}$ |  V   | (nbasis,nbasis) | 
|Electron Repulsion| $\sum^N_{i=1}\sum^N_{j>i} \frac{e^2}{\textbf{r}_{ij}}$   |Eri   | (nbasis,nbasis,nbasis,nbasis) |


## Operator

In [1]:
import numpy as np
from moha.basis import BasisSet
from moha.molecule import Molecule
from moha.hamiltonian import Hamiltonian

np.set_printoptions(precision=5,suppress=True)

geo = [[8,   0.000000000000,  -0.143225816552,   0.000000000000],
    ['h',   1.638036840407,   1.136548822547,  -0.000000000000],
    ["H",  -1.638036840407,   1.136548822547,  -0.000000000000]]

mol = Molecule.build(geo,pg=False)
orb = BasisSet.build(mol,'sto-3g.nwchem')

#ham = Hamiltonian.build(mol,orb)

In [2]:
# The overlap integral between two contracted Gaussian type orbital
from moha.hamiltonian.operator.overlap import Overlap
S_operator = Overlap()
cga = orb[0]
cgb = orb[6]
print(S_operator(cga,cgb))

# Build the overlap matrix
S_matrix = Overlap.build(orb)
print(S_matrix)

0.03840559992856808
[[ 1.       0.2367   0.      -0.       0.       0.03841  0.03841]
 [ 0.2367   1.       0.      -0.       0.       0.38614  0.38614]
 [ 0.       0.       1.       0.       0.       0.26844 -0.26844]
 [-0.      -0.       0.       1.       0.       0.20973  0.20973]
 [ 0.       0.       0.       0.       1.       0.       0.     ]
 [ 0.03841  0.38614  0.26844  0.20973  0.       1.       0.18176]
 [ 0.03841  0.38614 -0.26844  0.20973  0.       0.18176  1.     ]]


In [3]:
# The kinetic integral between two contracted Gaussian type orbital
from moha.hamiltonian.operator.kinetic import Kinetic
T_operator = Kinetic()
cga = orb[0]
cgb = orb[6]
print(T_operator.integral(cga,cgb))

# Build the kinetic matrix
T_matrix = Kinetic.build(orb)
print(T_matrix)

-0.008416383575885747
[[29.0032  -0.16801  0.      -0.       0.      -0.00842 -0.00842]
 [-0.16801  0.80813  0.      -0.       0.       0.07052  0.07052]
 [ 0.       0.       2.52873  0.       0.       0.14709 -0.14709]
 [-0.      -0.       0.       2.52873  0.       0.11492  0.11492]
 [ 0.       0.       0.       0.       2.52873  0.       0.     ]
 [-0.00842  0.07052  0.14709  0.11492  0.       0.76003 -0.00398]
 [-0.00842  0.07052 -0.14709  0.11492  0.      -0.00398  0.76003]]


In [4]:
# The nuclear attraction integral between two contracted Gaussian type orbital
from moha.hamiltonian.operator.nuclear_attraction import NuclearAttraction
V_operator = NuclearAttraction()
cga = orb[0]
cgb = orb[6]
print(V_operator(cga,cgb,mol))

# Build the nuclear attraction matrix
V_matrix = NuclearAttraction.build(mol,orb)
print(V_matrix)

-1.2316858773611625
[[-61.5806   -7.41082   0.       -0.01447   0.       -1.23169  -1.23169]
 [ -7.41082 -10.00907   0.       -0.17689   0.       -2.97723  -2.97723]
 [  0.        0.       -9.98755   0.        0.       -1.82224   1.82224]
 [ -0.01447  -0.17689   0.       -9.94404   0.       -1.47179  -1.47179]
 [  0.        0.        0.        0.       -9.87588   0.        0.     ]
 [ -1.23169  -2.97723  -1.82224  -1.47179   0.       -5.3002   -1.06717]
 [ -1.23169  -2.97723   1.82224  -1.47179   0.       -1.06717  -5.3002 ]]


In [9]:
from moha.hamiltonian.operator.multipole_moment import MultipoleMoment
mm = MultipoleMoment.build(orb,[1,2,0])
print(mm)

[[ 0.       0.       0.00369  0.       0.       0.00016 -0.00016]
 [ 0.       0.       0.27773  0.       0.       0.23425 -0.23425]
 [ 0.00369  0.27773  0.      -0.0851   0.       0.33356  0.33356]
 [ 0.       0.      -0.0851   0.       0.       0.22057 -0.22057]
 [ 0.       0.       0.       0.       0.       0.       0.     ]
 [ 0.00016  0.23425  0.33356  0.22057  0.       3.17987  0.     ]
 [-0.00016 -0.23425  0.33356 -0.22057  0.       0.      -3.17987]]


In [6]:
# The nuclear attraction integral between two contracted Gaussian type orbital
from moha.hamiltonian.operator.electron_repulsion import ElectronRepulsion
Eri_operator = ElectronRepulsion()
cga = orb[0]
cgb = orb[6]
print(Eri_operator(cga,cgb,cga,cgb))

# Build the nuclear attraction matrix
Eri_tensor = ElectronRepulsion.build(orb)
print(Eri_tensor.shape)

0.0036831079874511826
(7, 7, 7, 7)


## AO-MO Transformation

For the Post-Hartree-Fock methods, correlation energies are usually expressed in terms of MO-basis quantities (integrals, MO energies). Thus the transformation of the one- and two-electron integrals from the AO to the MO basis representation should be implementated.

With the optimized LCAO_MO coefficients, we can transform the operators from the atomic orbital basis to the molecular orbital basis.

- One-electron operator transformation:
\begin{equation}
h_{ij} = \sum_{\mu\nu}C_{\mu}^jC_{\nu}^i \langle\phi_{\mu}\vert\hat{h}\vert\phi_{\nu}\rangle
= \sum_{\mu\nu} C_{\mu}^j C_{\nu}^i F_{\mu\nu}
\end{equation}

In [7]:
C= np.random.rand(7,7)
T_ao = ham["T"]
T_mo = np.einsum("up, uv, vq -> pq", C, T_ao, C)

NameError: name 'ham' is not defined

- Two-electrons operator transformation:
\begin{equation}
\langle pq \vert rs\rangle = \sum_{\mu} C_{\mu}^p \left[
\sum_{\nu} C_{\nu}^q \left[
\sum_{\lambda} C_{\lambda}^r \left[
\sum_{\sigma} C_{\sigma}^s 
\langle\mu\nu\vert\lambda\sigma\rangle\right]\right]\right]
\end{equation}



* nmo Number of molecular orbitals

* nao Number of atomic orbitals

* natm Number of atoms

* nocc Number of occupied orbitals

* nvir Number of virtual(unoccupied) orbitals

* so Slice of occupied orbitals

* sv Slice of virtual(unoccupied) orbitals

* sa 全轨道分割


C 系数矩阵 

e 轨道能量 

Co 占据轨道系数矩阵 

Cv 未占轨道系数矩阵 

eo 占据轨道能量 

ev 未占轨道能量 

D 密度矩阵 

F_0_ao AO 基组 Fock 矩阵 

F_0_mo MO 基组 Fock 矩阵  (为对角阵)

H_0_ao AO 基组 Hamiltonian Core 矩阵 

H_0_mo MO 基组 Hamiltonian Core 矩阵 

eri0_ao AO 基组双电子互斥积分 

eri0_mo MO 基组双电子互斥积分 

mo_occ 轨道占据数

In [ ]:
eri_ao = ham["Eri"]
eri_mo = np.einsum("up, vq, uvkl, kr, ls -> pqrs", C, C, eri_ao, C, C)

In [ ]:
from scipy.special import factorial2 as fact2
print(fact2(1),fact2(-1),fact2(-3))